In [3]:
import numpy as np
import pandas as pd
def Rs_params_calc(m, t):
    n = pow(2, m) - 1
    k = n - 2 * t
    code_rate = round((k / n) * 100, 2)
    block_correction = round((t / n) * 100, 2)
    max_bits = n * k
    return [m, n, k, t, code_rate, block_correction, max_bits]

In [4]:
data = np.array([Rs_params_calc(4, 2), Rs_params_calc(8, 8)])
columns = ["m", "n", "k", "t", "Współczynnik wiadomości [%]", 'Zdolność korekcyjna [%]', 'Maksymalna ilość bitów']
frame = pd.DataFrame(data=data, columns=columns)
frame = frame.astype({'m':'int','n':'int', 'k':'int', 't':'int', 'Maksymalna ilość bitów':'int'})
frame

,m,n,k,t,Współczynnik wiadomości [%],Zdolność korekcyjna [%],Maksymalna ilość bitów
0,4,15,11,2,73.33,13.33,165
1,8,255,239,8,93.73,3.14,60945


# Parametry
### m - liczba bitów na jakich zapisany jest pojedyńczy symbol
### n - liczba symboli w całym słowie
### k - liczba symboli do zakodowania jako wiadomośc w słowie
### t - liczba poprawialnych symboli
### Współczynnik wiadomości - stosunek liczby symboli do zakodowania w słowie do liczby wszystich symboli w słowie
### Zdolność korekcyjna - stosunek liczby poprawialnych symboli do liczby wszystkich symboli w słowie
### Maksymalna ilość bitów - ile maksymalnie bitów może pomieścić jedna wiadmość